CSE 158/258 - Recommender Sys & Web Mining - FA24
# Assignment 2 Report

Andrew Choi (A69033628), Chanbin Na (AA18087468), Jonghee Chun (A69033997), Kenny Hwang (A99021639)

[Github Repo](https://github.com/cjychoi/cse258-fa24/tree/main)

## 1. Exploratory Dataset Analysis

**Dataset: Food.com Recipes and Interactions**\
Source: https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions

This dataset includes over 180K+ recipes and 700K+ recipe reviews spanning 18 years of user interactions and uploads on Food.com. For our analysis, we will utilize two files: **RAW_interactions.csv** and **RAW_recipes.csv**. We selected this dataset due to its rich and diverse information. Some of the interesting values included: 
```
user_id: Unique user ID values
recipe_id: Unique recipe ID values
rating: Rating given from user (0~5 scale)
review: Review of the recipe (text)
minutes: Minutes to prepare the recipe
tags: Food.com tags for each recipes
nutrition: Nutrition information (calories, total fat, sugar, sodium, protein, saturated fat)
n_steps: Number of steps in recipe
ingredients: List of ingredient names
n_ingredients: Number of ingredients
description: User-provided description
```

From the first glance, a lot of values seemed to have interesting correlations to each other, such as 'minutes', 'n_steps', and 'n_ingredients' may affect the user's 'rating' because it could be a burden to cook a complicated recipe and may lead to a lower rating.

We have summarized the properites of the dataset and was able to retrieve the following results:\
<img src="img/data_summary.png" width="280">
<img src="img/data_ratings.png" width="400">
<img src="img/data_ingredients.png" width="400">
<img src="img/data_recipe_sub.png" width="404">

### Analysis
- Dataset Characteristics:
    - The dataset is extensive, with over 230,000 recipes and 1.1 million interactions, making it robust for machine learning tasks like recommendation systems or rating predictions.
    - The average number of ingredients per recipe (around 9.55) suggests that most recipes are relatively simple, with a manageable number of ingredients.

- Rating Distribution:
    - The ratings are heavily skewed towards the highest score (5), which might indicate a bias in user reviews or a tendency for users to rate recipes they liked rather than disliked.
    - The small number of ratings below 4 suggests that poorly received recipes may either not be reviewed often or that users prefer not to leave negative feedback.

- Ingredients Popularity:
    - Common ingredients like salt, butter, sugar, and onion dominate the dataset, reflecting their foundational role in a wide variety of recipes.
    - These ingredients are versatile and likely to appear in both simple and complex recipes, which might impact model predictions for recipe similarity or ingredient importance.

- Temporal Trends:
    - Recipe submissions peaked around 2007, reflecting either a surge in platform popularity or increased engagement during that period.
    - The sharp decline after 2007 might be due to changes in platform usage, competition from other recipe-sharing platforms, or a reduction in user engagement.

- Insights for Predictive Modeling:
    - The skewed rating distribution could pose challenges for regression models, requiring careful handling, such as rebalancing or weighted loss functions, to mitigate bias.
    - The dominance of certain ingredients might mean they have less predictive power for ratings, as they are present in a large proportion of recipes.
    - Temporal trends could be factored into models to study how recipe popularity or user engagement changed over time, which might also correlate with rating patterns.


## 2. Predictive Task and Evaluation

## 3. Model Description

### Proposed Model:

Description of the model(s) used and why they were chosen.\
Explanation of how the model was implemented and optimized.

### Challenges

Major challenge that we had was determining the correlation between the features and the ratings.\
**TO-DO (Chanbin)**

## 4. Related Literature

## 5. Results and Conclusions

## 6. References

**Food.com Recipes and Interactions (Kaggle)**\
Crawled data from Food.com (GeniusKitchen) online recipe aggregator\
https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions?select=PP_recipes.csv


**Generating Personalized Recipes from Historical User Preferences**\
Bodhisattwa Prasad Majumder*, Shuyang Li*, Jianmo Ni, Julian McAuley
EMNLP, 2019\
https://www.aclweb.org/anthology/D19-1613/\
https://aclanthology.org/D19-1613.pdf
